In [46]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

df = pd.read_csv("synthetic.csv")

In [65]:
PROFMAX = 8

In [48]:
def entropie(df : pd.DataFrame):
    sum = 0
    ndata = df.shape[0]
    series = df["Class"].value_counts()
    #si on voulait parcourir les classes on passe par .index
    for nk in series:
        sum +=  (nk/ndata)*(math.log2(nk/ndata))
    return - sum

def gainatt(df : pd.DataFrame, a : str):
    e =  entropie(df)
    gain = 0
    split_value = 0
    partitions = [None, None]
    if a != "":
        for i in [0.25, 0.5, 0.75]:
            q = df.quantile(i)[a]
            p0test = df[df[a] < q]
            p1test = df[df[a] >= q]
            gaintest = e - ( (len(p0test) * entropie(p0test) / len(df)) + (len(p1test) * entropie(p1test) / len(df)) )
            if ( gaintest > gain ):
                split_value = q
                partitions[0] = p0test
                partitions[1] = p1test
                gain = gaintest
                break
    return(gain, split_value, partitions)

def gain(df : pd.DataFrame, atts : list[str]):
    gain = 0.
    att = ""
    res = (0, 0, [None, None])
    for i in atts:
        res = gainatt(df, i)
        if(res[0] >= gain):
            gain = res[0]
            att = i
    return (att,)+gainatt(df,att)


class Noeud(object):
    def __init__(self, vsplit =0, att=None, fg=None, fd=None, pred : pd.Series =None, leaf : bool =True):
        self.vsplit = vsplit
        self.att = att
        self.fg = fg
        self.fd = fd
        self.pred = pred
        self.leaf = leaf
    def node_result(self, spacing):
        string = ""
        for x,y in self.pred.items():
            string += spacing + " Class " + str(x) + " Count : " + str(y) + "\n"
        string += spacing
        return string

def construction_arbre(df : pd.DataFrame, label : str, atts : list[str], prof : int):
    att, g, split, parts = gain(df, atts)
    prediction = df[label].value_counts()
    if(prof >= PROFMAX or len(atts) == 0 or g == 0):
        return Noeud(pred = prediction, leaf=True)
    att_rest = [x for x in atts if x != att]
    fg = construction_arbre(parts[0], label, att_rest, prof+1)
    fd = construction_arbre(parts[1], label, att_rest, prof+1)
    return Noeud(split, att, fg,fd, prediction, leaf=False)
    
def print_tree(node : Noeud, spacing : str =" "):
    if node is None:
        return
    if node.leaf:
        print(node.node_result(spacing))
        return
    print("{}[Attribute: {} Split value: {}]".format(spacing, node.att, node.vsplit))
    print(spacing + "> True")
    print_tree(node.fg , spacing + "-")
    print(spacing + "> False")
    print_tree(node.fd , spacing + "-")
    return

def inference(instance : pd.DataFrame, node : Noeud, a : str):
    if(node.leaf):
        # Affichage des pourcentages des prédictions
        # sum = 0
        # maxcount = 0
        # for i in range(len(node.pred)):
        #     sum += node.pred.values[i]
        # for i in range(len(node.pred)):
        #     print("Class ", node.pred.index[i], " : ", node.pred.values[i]*100/sum,"%")
        return node.pred.index[0]
    else:
        v = instance[a]
        if(v < node.vsplit):
            return inference(instance, node.fg, node.fg.att)
        else:
            return inference(instance, node.fd, node.fd.att)

In [70]:
cols = df.columns.values.tolist()

features = cols[0:14] 
label = cols[14]
X = df[features]
y = df[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

'Class'

In [50]:
df_train = pd.concat([X_train, y_train],axis=1)
df_test = pd.concat([X_test, y_test],axis=1)

df_actual = pd.DataFrame(y_test)
df_actual.to_csv("arbre/y_test.csv",index=False, header=False)

In [66]:
arbre = construction_arbre(df_train, label, features, 0)

In [67]:
preds_class :list[int] = []
for i in range(len(df_test)):
    preds_class += [inference(df_test.iloc[i], arbre, arbre.att)]

df_preds = pd.DataFrame(data=preds_class, columns=[0,])

filnename = "arbre/preds_arbre"
filnename += str(PROFMAX) + ".csv"

df_preds.to_csv(filnename, index=False, header=False)